In [26]:
import pandas as pd
import os
import datetime
from datetime import date
import numpy as np
import sys
import pickle
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
import jellyfish
from sklearn.cluster import KMeans

In [27]:
def saveObj(obj, name ):
    with open('../objects/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [28]:
def loadObj(name):
    with open('../objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [29]:
def preprocess(data):
    #Calculating age from dob
    now  = pd.Timestamp(datetime.datetime.now())
    data['dob'] = data['dob']
    data['dob'] = pd.to_datetime(data['dob'], format='%d/%m/%y')
    data['dob'] = data['dob'].where(data['dob'] < now, data['dob'] -  np.timedelta64(100, 'Y'))
    data['age'] = (now - data['dob']).astype('<m8[Y]')
    
    #dropping dob column
    data2 = data.drop('dob', 1)
    
    #Gender assignment
    data2.rename(columns={'gn': 'gender'}, inplace=True)
    for i,gen in enumerate(data2.gender):
        if gen=="F":
            data2.gender[i] = 1
        elif gen=="M":
            data2.gender[i] = 0
        else:
            data2.gender[i] = -1
    
    #size of data
    l = len(data2)
    return (data2, l)

In [2]:
def clustering(data, data_size):
    cluster = []
    for i in range(0,data_size):
        temp = []
        flag = 1
        x = data["fn"][i] + ' ' + data["ln"][i]
        for j in range(len(cluster)):
            if x in cluster[j]:
                flag=0
        if flag==1:
            temp.append(x)
            for j in range(i+1,data_size):
                if (data["age"][i]==data["age"][j] and data["gender"][i]==data["gender"][j] and data["ln"][i]==data["ln"][j]):
                    y = data["fn"][j] + ' ' + data["ln"][j]
                    temp.append(y)
            cluster.append(temp)
    return cluster